In [1]:
!nvidia-smi

Sun Aug 13 06:08:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.263 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [3]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.0/auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl

In [4]:
!pip install -qqq auto_gptq-0.4.0+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [5]:
!mkdir pdfs

In [6]:
!gdown 1v-Rn1FVU1pLTAQEgm0N9oB6cExMoebZr -O pdfs/tesla-earnings-report.pdf
!gdown 1Xc890jrQvCExAkryVWAttsv1DBLdVefN -O pdfs/nvidia-earnings-report.pdf
!gdown 1Epz-SQ3idPpoz75GlTzzomag8gplzLv8 -O pdfs/meta-earnings-report.pdf

Downloading...
From: https://drive.google.com/uc?id=1v-Rn1FVU1pLTAQEgm0N9oB6cExMoebZr
To: /content/pdfs/tesla-earnings-report.pdf
100% 345k/345k [00:00<00:00, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Xc890jrQvCExAkryVWAttsv1DBLdVefN
To: /content/pdfs/nvidia-earnings-report.pdf
100% 414k/414k [00:00<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Epz-SQ3idPpoz75GlTzzomag8gplzLv8
To: /content/pdfs/meta-earnings-report.pdf
100% 183k/183k [00:00<00:00, 87.6MB/s]


In [7]:
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline

## Data

In [8]:
!rm -rf "db"

In [9]:
loader = PyPDFDirectoryLoader("pdfs")

In [10]:
docs = loader.load()

In [11]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)

In [13]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

## Llama 2 13B

In [14]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device="cuda:0",
    quantize_config=None,
)

In [15]:
!nvidia-smi

Sun Aug 13 06:13:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    39W /  70W |   8483MiB / 15360MiB |     94%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [17]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [18]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausal

In [19]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [20]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [21]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [22]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat with Multiple PDFs

In [23]:
result = qa_chain("What is the per share revenue for Meta during 2023?")

 Based on the information provided in the press release, the per share revenue for Meta during 2023 was $2.98.


In [24]:
len(result["source_documents"])

2

In [25]:
print(result["source_documents"][0].page_content)

Meta Reports Second  Quarter 2023  Results
MENLO PARK, Calif. – July 26, 2023  – Meta Platforms, Inc. (Nasdaq: META) today reported financial results for the 
quarter ended June 30, 2023 .
"We had a good quarter. We continue to see strong engagement across our apps and we have the most exciting roadmap I've 
seen in a while with Llama 2, Threads, Reels, new AI products in the pipeline, and the launch of Quest 3 this fall," said Mark 
Zuckerberg, Meta founder and CEO.
Second Quarter 2023  Financial Highlights
Three Months Ended June 30,
In millions, except percentages and per share amounts 2023 2022 % Change
Revenue $ 31,999 $ 28,822 11%
Costs and expenses  22,607  20,464 10%
Income from operations $ 9,392 $ 8,358 12%
Operating margin  29 %  29 %
Provision for income taxes $ 1,505 $ 1,499 —%
Effective tax rate  16 %  18 %
Net income $ 7,788 $ 6,687 16%
Diluted earnings per share (EPS) $ 2.98 $ 2.46 21%
Second Quarter 2023  Operational and Other Financial Highlights


In [26]:
result = qa_chain("What is the per share revenue for Tesla during 2023?")

 I can answer this question based on the information provided in the financial statements.

According to the Consolidated Statements of Operations, the total revenues for Tesla in 2023 were $24,927 million.

To calculate the per share revenue, we need to divide the total revenues by the number of shares outstanding. However, the financial statements do not provide the number of shares outstanding for 2023.

Therefore, I cannot accurately determine the per share revenue for Tesla in 2023.


In [27]:
result = qa_chain("What is the per share revenue for Nvidia during 2023?")

 I cannot determine the per share revenue for Nvidia during 2023 based on the information provided. The information only provides data up to April 30, 2023, and does not include any information about the second quarter of fiscal year 2024 or beyond. Additionally, the information provided does not include any details about the company's revenue or earnings per share.


In [28]:
result = qa_chain("What is the per share revenue for Nvidia?")

 Based on the information provided, the per share revenue for Nvidia is $0.83. This is calculated by dividing the total revenue ($7,192 million) by the weighted average shares outstanding (basic and diluted; 2,470 million).


In [29]:
print(result["source_documents"][1].page_content)

PART I. FINANCIAL INFORMATION
ITEM 1.  FINANCIAL STATEMENTS (UNAUDITED)
NVIDIA CORPORATION AND SUBSIDIARIES
CONDENSED CONSOLIDATED STATEMENTS OF INCOME
(In millions, except per share data)
(Unaudited)
 Three Months Ended
 April 30, May 1,
2023 2022
Revenue $ 7,192 $ 8,288 
Cost of revenue 2,544 2,857 
Gross profit 4,648 5,431 
Operating expenses   
Research and development 1,875 1,618 
Sales, general and administrative 633 592 
Acquisition termination cost — 1,353 
Total operating expenses 2,508 3,563 
Income from operations 2,140 1,868 
Interest income 150 18 
Interest expense (66) (68)
Other, net (15) (13)
Other income (expense), net 69 (63)
Income before income tax 2,209 1,805 
Income tax expense 166 187 
Net income $ 2,043 $ 1,618 
Net income per share:
Basic $ 0.83 $ 0.65 
Diluted $ 0.82 $ 0.64 
Weighted average shares used in per share computation:
Basic 2,470 2,506 
Diluted 2,490 2,537 
See accompanying Notes to Condensed Consolidated Financial Statements.
3


In [30]:
result = qa_chain("What is the estimated YOY revenue for Meta during 2023?")

 Based on the information provided in the press release, the estimated YOY revenue for Meta during 2023 is 11%. This can be calculated by comparing the revenue for the second quarter of 2023 ($31,999 million) to the revenue for the second quarter of 2022 ($28,822 million).


In [31]:
result = qa_chain("What is the estimated YOY revenue for Tesla during 2023?")

 Based on the information provided in the financial statements, I can provide an estimate of Tesla's year-over-year (YOY) revenue for 2023.

According to the financial statements, the total revenues for the three months ended June 30, 2023 were $24,927 million, which represents a 48% increase compared to the same period in 2022. Additionally, the energy generation and storage segment reported a 74% increase in revenue for the three months ended June 30, 2023 compared to the same period in 2022.

Assuming the trend of increasing revenue continues for the remaining nine months of 2023, we can estimate the full-year YOY revenue growth for Tesla in 2023 as follows:

Full-year YOY revenue growth = (3 months YOY revenue growth x 9 months remaining in 2023) + (6 months YOY revenue growth x 3 months remaining in 2023)

Using the financial statements, we can calculate the YOY revenue growth for each of the three months and six months ended June 30, 2023, as follows:

Three months ended June 30,

In [32]:
result = qa_chain("What is the estimated YOY revenue for Nvidia during 2023?")

 Based on the information provided, there is no mention of estimated revenue for Nvidia during 2023. The information provided only discusses financial performance for the first quarter of fiscal year 2024, which ended on May 1, 2023. Therefore, I cannot provide an estimate of YOY revenue for Nvidia during 2023.


In [33]:
result = qa_chain(
    "Which company is more profitable during 2023 Meta, Nvidia or Tesla and why?"
)

 Based on the information provided in the press release, it appears that Meta (formerly Facebook) had higher net income in the second quarter of 2023 compared to both Nvidia and Tesla. According to the financial highlights, Meta's net income was $7,788 million, while Nvidia's net income was $2,047 million and Tesla's net income was $1,832 million. Therefore, Meta was the most profitable company in the second quarter of 2023.

The reason for this may be due to Meta's diverse range of products and services, including its social media platforms, virtual reality technology, and e-commerce offerings, which generate a wide range of revenue streams and help to offset any challenges or declines in specific areas. Additionally, Meta has made significant investments in artificial intelligence and machine learning, which are expected to drive future growth and profitability.


In [34]:
result = qa_chain(
    "In which company would you invest to maximize your profits for the long term (10+ years)?"
)

 I cannot provide personalized financial advice or make specific investment recommendations. However, based on the information provided, it appears that the company has experienced significant revenue growth over the past year, with a 19% increase in revenue from the same period last year and a 13% decrease from the same period two years ago. The company also reported net income of $2.043 billion, representing a 44% increase from the same period last year. Additionally, the company has a strong cash position, with $23.08 billion in cash and cash equivalents and investments as of the end of the second quarter of fiscal year 2023.

However, without more information about the company's financial health, industry trends, competitors, and other factors, it is difficult to determine whether investing in this company would be a good long-term decision. It is important to conduct thorough research and analysis before making any investment decisions.


## References

- [Tesla Quarterly Report (Jul 21, 2023)](https://ir.tesla.com/_flysystem/s3/sec/000095017023033872/tsla-20230630-gen.pdf)
- [Meta Q2 2023 Earnings (Jul 26, 2023)](https://s21.q4cdn.com/399680738/files/doc_financials/2023/q2/Meta-06-30-2023-Exhibit-99-1-FINAL.pdf)
- [Nvidia Fiscal Q1 2024](https://s201.q4cdn.com/141608511/files/doc_financials/2024/q1/ecefb2b2-efcb-45f3-b72b-212d90fcd873.pdf)